# A Hybrid Algorithm for Solving the 3D Bin Packing Problem

The Jupyter notebook contains the neccessary elements to run the test to obtain solution to the 3D Bin Packing Problem using a Hybrid Algorithm. 

## Initial Configuration

All the params defined on the paper, are configure on the file INSTANCE_PARAM.py. As reviewer, you can access to them and modify accord your neccessity (don't forget return the configuration to the original value)



In [ ]:
from base.INSTACE_PARAM import (
    min_fr,                     # Min block generation size
    max_bl,                     # Max block generation size
    MAX_ITER,
    instances_name,             # Instance Name, Use 'cg' for Thapasuwan instances
    bsg_time,                   # Execution time for solving using BSG
    r_param,                    # R param to generate bin
    max_no_improvements,        # Improve
    swaps,                      # Number of oportunities to make a swaps between bins
    n_runs,                     # Number of runs for the algorithm
    min_bin,                    # Min bin size
    VERBOSE,                    # Modo verbose
)

# This params currently it's not neccessary for a test
ssh = None

# Define extra args
extra_args = f"--max_bl={max_bl} --min_fr={min_fr} --show_layout"
if bsg_time == 0:
    extra_args += " --greedy_only"

# Imprime todos los parametros que son importados
print(f"Instances: {instances_name}")
print(
    f"min_fr={min_fr}, max_bl={max_bl}, bsg_time={bsg_time}, r_param={r_param}, n_runs={n_runs}, max_no_improvements={max_no_improvements}"
)
print(f"Extra args: {extra_args}")

## Data Instance

On the file INSTANCE_PARAM.py, you can find the constant "instances_name", a string defined as the name of the evaluated instance to resolve. This instance has 3 different values currently, which are:
* martello : Martello instances
* dl: Another instances of Martello, that contains problem with a bigger size
* cg: Thapatsuwan instances


In [ ]:
from base.Heuristics.dataset import DatasetLoader

dataloader = DatasetLoader(instances_name)
instance_files = dataloader.instances_list

print(f"Running BSG with {n_runs} runs on {instances_name} instances")
print(f'Instances List: {instance_files}')
print(f"Total: {len(instance_files)}")

In [ ]:
from base.Heuristics.MCLP import MCLP
from base.baseline.bin import bin
from statistics import mean
import math
import timeit

print("Instance Name, Best Solution, Mean Solution, Minimal , Mean Time")
# Execute the algorithm for each instance
for filename in instance_files:
    
    times = []
    sols = []
    best_sols = []

    tot_bins = 0
    min_bins = min_bin

    # Load instance info
    L, W, H, _boxes, id2box = dataloader.get_instance(filename=filename)

    total_vol = bin.get_vol_by_boxes_group(_boxes)
    lb = math.ceil(total_vol)

    mclp_instance = MCLP(ssh, L, W, H, id2box, VERBOSE)

    # Running experiments n_runs times
    for k in range(n_runs):
        start_time = timeit.default_timer()
        boxes = _boxes.copy()

        # Generate initial solution(
        best_solution = mclp_instance.generate_candidate_solution(
        boxes, r_param, bsg_time, extra_args)

        # Aplication Swaps sobre una solucion mejor que la propuesta
        if swaps == "--swaps" and len(best_solution) != lb:
            best_solution = mclp_instance.random_swaps(
                best_solution=best_solution,
                max_iter=MAX_ITER,
                extra_args=extra_args,
                lb=lb,
                max_no_improvements=max_no_improvements,
                bsg_time=bsg_time
            )
        else:
            if len(best_solution) == lb:
                print(
                    "The algorithm gots lower bound on the Generate candidate solution")
        tot_bins += len(best_solution)

        # Store the min solution of the run
        if len(best_solution) < min_bins:
            min_bins = len(best_solution)

        resolution_time = timeit.default_timer() - start_time

        # Store results
        times.append(resolution_time)

    sols.append(tot_bins / n_runs)
    best_sols.append(min_bins)

    print(filename, ":", mean(best_sols),
          mean(sols), min(sols), mean(times))

